In [46]:
pip install pipwin

You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [47]:
pip install PyAudio-0.2.11-cp310-cp310-win_amd64.whl

Note: you may need to restart the kernel to use updated packages.


ERROR: PyAudio-0.2.11-cp310-cp310-win_amd64.whl is not a supported wheel on this platform.
You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [48]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [49]:
pip install tqdm

You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [50]:
import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv("C:/Age Gender Detection/Voice Recognition/balanced-all.csv")
df.head()

,filename,gender
0,data/cv-other-train/sample-069205.npy,female
1,data/cv-valid-train/sample-063134.npy,female
2,data/cv-other-train/sample-080873.npy,female
3,data/cv-other-train/sample-105595.npy,female
4,data/cv-valid-train/sample-144613.npy,female


In [52]:
df.tail()

,filename,gender
66933,data/cv-valid-train/sample-171098.npy,male
66934,data/cv-other-train/sample-022864.npy,male
66935,data/cv-valid-train/sample-080933.npy,male
66936,data/cv-other-train/sample-012026.npy,male
66937,data/cv-other-train/sample-013841.npy,male


In [53]:
# get total samples
n_samples = len(df)
# get total male samples
n_male_samples = len(df[df['gender'] == 'male'])
# get total female samples
n_female_samples = len(df[df['gender'] == 'female'])
print("Total samples:", n_samples)
print("Total male samples:", n_male_samples)
print("Total female samples:", n_female_samples)

Total samples: 66938
Total male samples: 33469
Total female samples: 33469


In [54]:
label2int = {
    "male": 1,
    "female": 0
}

def load_data(vector_length=128):
    """A function to load gender recognition dataset from `data` folder
    After the second run, this will load from results/features.npy and results/labels.npy files
    as it is much faster!"""
    # make sure results folder exists
    if not os.path.isdir("C:/Age Gender Detection/Voice Recognition/results"):
        os.mkdir("C:/Age Gender Detection/Voice Recognition/results")
    # if features & labels already loaded individually and bundled, load them from there instead
    if os.path.isfile("C:/Age Gender Detection/Voice Recognition/results/features.npy") and os.path.isfile("C:/Age Gender Detection/Voice Recognition/results/labels.npy"):
        X = np.load("C:/Age Gender Detection/Voice Recognition/results/features.npy")
        y = np.load("C:/Age Gender Detection/Voice Recognition/results/labels.npy")
        return X, y
    # read dataframe
    df = pd.read_csv("C:/Age Gender Detection/Voice Recognition/balanced-all.csv")
    # get total samples
    n_samples = len(df)
    # get total male samples
    n_male_samples = len(df[df['gender'] == 'male'])
    # get total female samples
    n_female_samples = len(df[df['gender'] == 'female'])
    print("Total samples:", n_samples)
    print("Total male samples:", n_male_samples)
    print("Total female samples:", n_female_samples)
    # initialize an empty array for all audio features
    X = np.zeros((n_samples, vector_length))
    # initialize an empty array for all audio labels (1 for male and 0 for female)
    y = np.zeros((n_samples, 1))
    for i, (filename, gender) in tqdm.tqdm(enumerate(zip(df['filename'], df['gender'])), "Loading data", total=n_samples):
        features = np.load(filename)
        X[i] = features
        y[i] = label2int[gender]
    # save the audio features and labels into files
    # so we won't load each one of them next run
    np.save("C:/Age Gender Detection/Voice Recognition/results/features", X)
    np.save("C:/Age Gender Detection/Voice Recognition/results/labels", y)
    return X, y

In [55]:
def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }

In [56]:
# load the dataset
X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)

In [57]:
def create_model(vector_length=128):
    """5 hidden dense layers from 256 units to 64, not the best model."""
    model = Sequential()
    model.add(Dense(256, input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 0 means female, 1 means male
    model.add(Dense(1, activation="sigmoid"))
    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    # print summary of the model
    model.summary()
    return model

In [58]:
# construct the model
model = create_model()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               65792     
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 128)              

In [59]:
# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100
# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
# save the model to a file
model.save("C:/Age Gender Detection/Voice Recognition/voice_recognition_model.h5")

In [ ]:
# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Evaluating the model using 6694 samples...
Loss: 0.2275
Accuracy: 91.66%


In [ ]:
import librosa
import numpy as np

def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

In [ ]:
pip install PySoundFile

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
pip install --upgrade librosa audioread

Requirement already up-to-date: librosa in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (0.10.1)
Requirement already up-to-date: audioread in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (3.0.1)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
pip install --upgrade librosa PySoundFile audioread


Requirement already up-to-date: librosa in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (0.10.1)
Requirement already up-to-date: PySoundFile in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (0.9.0.post1)
Requirement already up-to-date: audioread in c:\users\user\appdata\local\programs\python\python37\lib\site-packages (3.0.1)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import librosa

# Load the saved/trained model
model_path = "C:/Age Gender Detection/Voice Recognition/voice_recognition_model.h5"
model = load_model(model_path)

if model is None:
    print("Error: Failed to load the model")
    exit(1)

def extract_feature(file, mel=True):
    try:
        # Load audio file
        y, sr = librosa.load(file, sr=None)

        # Extract features
        if mel:
            feature = librosa.feature.mfcc(y=y, sr=sr)
        else:
            feature = librosa.feature.melspectrogram(y=y, sr=sr)

        return feature
    except Exception as e:
        print(f"Error extracting features from the file: {e}")
        return None

# Path to the WAV file
file = input("Enter the path to the WAV file: ")

if not os.path.isfile(file):
    print(f"Error: File '{file}' does not exist")
    exit(1)

# Extract features from the WAV file
features = extract_feature(file)

if features is None:
    print("Feature extraction failed. Please check the WAV file.")
    exit(1)

# Reshape the features if necessary
# features = features.reshape(...)  # Reshape as needed by your model

# Perform inference (prediction)
# You'll need to modify this part according to your model's input and output
# For example, if your model expects input shape (batch_size, num_features) and outputs probabilities for each class
# predictions = model.predict(features)

# Placeholder for prediction logic
male_prob = 0.5  # Placeholder for male probability
female_prob = 1 - male_prob  # Placeholder for female probability
gender = "male" if male_prob > female_prob else "female"  # Placeholder for gender prediction

# Display the result
print("Result:", gender)
print(f"Probabilities::: Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")


Result: female
Probabilities::: Male: 50.00%    Female: 50.00%


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import librosa

# Load the saved/trained model
model_path = "C:/Age Gender Detection/Voice Recognition/voice_recognition_model.h5"
model = load_model(model_path)

if model is None:
    print("Error: Failed to load the model")
    exit(1)

def extract_feature(file, mel=True, n_mfcc=20):
    try:
        # Load audio file
        y, sr = librosa.load(file, sr=None)

        # Extract features
        if mel:
            feature = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        else:
            feature = librosa.feature.melspectrogram(y=y, sr=sr)

        # Pad or truncate the features to ensure consistent length along the time axis
        target_length = 128
        if feature.shape[1] < target_length:
            feature = np.pad(feature, ((0, 0), (0, target_length - feature.shape[1])), mode='constant')
        elif feature.shape[1] > target_length:
            feature = feature[:, :target_length]

        # Add channel dimension
        feature = np.expand_dims(feature, axis=-1)

        return feature
    except Exception as e:
        print(f"Error extracting features from the file: {e}")
        return None

# Path to the WAV file
file = input("Enter the path to the WAV file: ")

if not os.path.isfile(file):
    print(f"Error: File '{file}' does not exist")
    exit(1)

# Extract features from the WAV file
features = extract_feature(file)

if features is None:
    print("Feature extraction failed. Please check the WAV file.")
    exit(1)

# Perform inference (prediction)
predictions = model.predict(features)

# Interpret prediction
prediction = predictions[0][0]  # Assuming it's a single value representing probability for one class

# Determine gender based on prediction probability
gender = "male" if prediction >= 0.5 else "female"

# Display the result
print("Result:", gender)
print(f"Probability: {prediction*100:.2f}%")


1/1 [==============================] - 0s 126ms/step
Result: male
Probability: 100.00%


In [60]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import librosa

# Load the saved/trained model
model_path = "C:/Age Gender Detection/Voice Recognition/voice_recognition_model.h5"
model = load_model(model_path)

if model is None:
    print("Error: Failed to load the model")
    exit(1)

def extract_feature(file, mel=True, n_mfcc=20):
    try:
        # Load audio file
        y, sr = librosa.load(file, sr=None)

        # Extract features
        if mel:
            feature = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        else:
            feature = librosa.feature.melspectrogram(y=y, sr=sr)

        # Pad or truncate the features to ensure consistent length along the time axis
        target_length = 128
        if feature.shape[1] < target_length:
            feature = np.pad(feature, ((0, 0), (0, target_length - feature.shape[1])), mode='constant')
        elif feature.shape[1] > target_length:
            feature = feature[:, :target_length]

        # Add channel dimension
        feature = np.expand_dims(feature, axis=-1)

        return feature
    except Exception as e:
        print(f"Error extracting features from the file: {e}")
        return None

# Path to the WAV file
file = input("Enter the path to the WAV file: ")

if not os.path.isfile(file):
    print(f"Error: File '{file}' does not exist")
    exit(1)

# Extract features from the WAV file
features = extract_feature(file)

if features is None:
    print("Feature extraction failed. Please check the WAV file.")
    exit(1)

# Perform inference (prediction)
predictions = model.predict(features)

# Interpret prediction
prediction = predictions[0][0]  # Assuming it's a single value representing probability for one class

# Determine gender based on prediction probability
gender = "male" if prediction >= 0.5 else "female"

# Display the result without printing probability
print("Result:", gender)


1/1 [==============================] - 0s 107ms/step
Result: male


In [11]:
import os
import numpy as np
from tensorflow.keras.models import load_model
import librosa

# Load the saved/trained model
model_path = "C:/Age Gender Detection/Voice Recognition/voice_recognition_model.h5"
model = load_model(model_path)

if model is None:
    print("Error: Failed to load the model")
    exit(1)

def extract_feature(file, mel=True, n_mfcc=20):
    try:
        # Load audio file
        y, sr = librosa.load(file, sr=None)

        # Extract features
        if mel:
            feature = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
        else:
            feature = librosa.feature.melspectrogram(y=y, sr=sr)

        # Pad or truncate the features to ensure consistent length along the time axis
        target_length = 128
        if feature.shape[1] < target_length:
            feature = np.pad(feature, ((0, 0), (0, target_length - feature.shape[1])), mode='constant')
        elif feature.shape[1] > target_length:
            feature = feature[:, :target_length]

        # Add channel dimension
        feature = np.expand_dims(feature, axis=-1)

        return feature
    except Exception as e:
        print(f"Error extracting features from the file: {e}")
        return None

# Path to the WAV file
file = input("Enter the path to the WAV file: ")

if not os.path.isfile(file):
    print(f"Error: File '{file}' does not exist")
    exit(1)

# Extract features from the WAV file
features = extract_feature(file)

if features is None:
    print("Feature extraction failed. Please check the WAV file.")
    exit(1)

# Perform inference (prediction)
predictions = model.predict(features)

# Interpret prediction
prediction = predictions[0][0]  # Assuming it's a single value representing probability for one class

# Determine gender based on prediction probability
gender = "male" if prediction >= 0.5 else "female"

# Display the result without printing probability
print("Result:", gender)


1/1 [==============================] - 0s 142ms/step
Result: female
